## Purpose of this notebook

- Examine similarities and differences between different raters' reliability report grades
- Calculate Cohen's kappa between pairs of raters
- Identify and print reports where raters strongly disagree (grade of 2 vs grade of 0)

## How to use this notebook

- Run each of the cells in order. Make sure you run Cell 01 first.
- Cell 02 can be used to see how many reports have been graded since a date (YYYY-MM-DD format)
- Cells 03-04 get the proc_ord_id values unique to each report and the names of the persons who have graded reliability reports.
- Cell 05 can be used to examine the reports a pair of graders disagree on
- Cells 06 will release examined reports back to a specified grader for regrading
- Cell 07 can be used to examine and regrade reports marked with a -1 flag

In [ ]:
# Cell 01: load libraries
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the code directory to sys.path
sys.path.append(os.path.join(os.path.dirname(current_dir), 'code'))

from reliabilityLib import *
from reportMarkingFunctions import *
from google.cloud import bigquery # SQL table interface on Arcus
import pandas
import numpy
import random
import matplotlib.pyplot as plt
import datetime

client = bigquery.Client()
backup_grader_table()

## Grades Over Time

In [ ]:
# Cell 02:
get_grade_counts_since("2024-10-01")

## Grader agreement on reliability reports

In [ ]:
# Cell 03: Get the list of proc_ord_id values used to identify the reliability reports
procIds = get_reliability_proc_ord_ids()

In [ ]:
# Cell 04: Compare the reliability reports for the users we want to evaluate
graders = ['Grader 1', 
           'Grader 2']

# Metric options: "disagreement", "kappa", "kappa2vAll", "kappa0vAll"
df = calculate_metric_for_graders(graders, "kappa")
print()
print(df)

In [ ]:
# Cell 05: 
# This is the cell where you can look at the disagreement reports for each pair of users
grader1 = "Grader 1"
grader2 = "Grader 2"
procIds = get_reliability_proc_ord_ids()
grades1 = get_reports_for_user(grader1, procIds)
grades2 = get_reports_for_user(grader2, procIds)
disagreement = identify_disagreement_reports(grades1, grades2)
print(len(disagreement))

calc_kappa(grades1, grades2)

print_disagreement_reports(disagreement, grades1, grades2)

In [ ]:
# Cell 06: release a set of your reports back into your queue 
# ONLY USE THIS IF YOU'RE CERTAIN
# release_reports(grader2, disagreement)

## Examine flagged reports

In [ ]:
# Cell 07: regrade skipped reports
# client: A bigquery client object (created in Cell 01)
# skippedGrader: A string of the grader's name (leave blank to review all flagged reports)
skippedGrader = "Grader Name"
regrade_skipped_reports(client, grader=skippedGrader)

## For clinician review only

In [ ]:
# Cell 08: clinician regrade skipped reports
# client: A bigquery client object (created in Cell 01)
# skippedGrader: A string of the grader's name (leave blank to review all flagged reports)
regrade_skipped_reports(client, flag=-2)